In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('2015.csv')

### Course of Action
1) Fixing Airport data and merging with the main Airline dataset.
2) Preprocessing for Regression Model(Aim- Predict departure delay)
3) Preprocessing for Classification Model(Aim- Predict if the airline will delay or not)

In [3]:
# Extracting important features and imputing the null values with mean
Airport_Weather = pd.read_csv('airport_weather_dataset_Updated.csv')
Airport_Weather = Airport_Weather[['Date', 'tavg', 'prcp', 'wspd', 'Airport Code']]
Airport_Weather['tavg'] = Airport_Weather['tavg'].fillna(np.mean(Airport_Weather['tavg']))
Airport_Weather['prcp'] = Airport_Weather['prcp'].fillna(np.mean(Airport_Weather['prcp']))
Airport_Weather['wspd'] = Airport_Weather['wspd'].fillna(np.mean(Airport_Weather['wspd']))
Airport_Weather.isnull().sum()

Date            0
tavg            0
prcp            0
wspd            0
Airport Code    0
dtype: int64

In [4]:
# Extracting the important columns from the Airline dataset
df_final1 = df[['FL_DATE', 'OP_CARRIER', 'ORIGIN', 'CRS_DEP_TIME', 'DEP_DELAY']]

In [5]:
df_final1.isnull().sum()

FL_DATE             0
OP_CARRIER          0
ORIGIN              0
CRS_DEP_TIME        0
DEP_DELAY       86153
dtype: int64

In [32]:
# Removing records where the target variable(dep_delay) contains null values
df_final1.dropna(subset='DEP_DELAY', inplace=True)

In [34]:
# Reformatting the datatype of Date and flight date column; Then merging both datasets(Airline and Weather)
Airport_Weather['Date'] = pd.to_datetime(Airport_Weather['Date'], format='%Y-%m-%d')
df_final1['FL_DATE'] = pd.to_datetime(df_final1['FL_DATE'], format='%Y-%m-%d')
df_final1 = pd.merge(df_final1, Airport_Weather, left_on=['ORIGIN', 'FL_DATE'], right_on=['Airport Code', 'Date'])

In [36]:
(df_final1['DEP_DELAY'] <= 0).value_counts(normalize=True)*100

DEP_DELAY
True     62.80722
False    37.19278
Name: proportion, dtype: float64

We see a slight imbalance, can use SMOTE

# Preprocessing for regression

### Case 1:
#### Considering only +ve dep_delay values and thus removing values < 0
#### Aim - To predict departure delay(numerical)

### Train

In [80]:
# Delay above 180 minutes become unrealistic and 
df_final1 = df_final1[(df_final1['DEP_DELAY'] >= 0) & (df_final1['DEP_DELAY'] < 180)]

In [50]:
#Since we will be performing target encoding and to prevent data leakage, we will do the preprocessing separately for train and test data
from sklearn.model_selection import train_test_split
y = df_final1['DEP_DELAY']
X = df_final1.drop('DEP_DELAY', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
train_data = X_train.copy()
train_data['DEP_DELAY'] = y_train

In [52]:
train_data.head()

,FL_DATE,ORIGIN,CRS_DEP_TIME,OP_CARRIER_AS,OP_CARRIER_B6,OP_CARRIER_DL,OP_CARRIER_EV,OP_CARRIER_F9,OP_CARRIER_HA,OP_CARRIER_MQ,OP_CARRIER_NK,OP_CARRIER_OO,OP_CARRIER_UA,OP_CARRIER_US,OP_CARRIER_VX,OP_CARRIER_WN,Date,tavg,prcp,wspd,Airport Code,DEP_DELAY
1276095,2015-03-27,SJU,1646,False,True,False,False,False,False,False,False,False,False,False,False,False,2015-03-27,26.6,0.0,16.6,SJU,16.0
2100329,2015-05-18,PIT,630,False,False,False,False,False,False,False,False,False,False,False,False,True,2015-05-18,22.7,5.8,9.4,PIT,0.0
3830405,2015-09-02,IAH,910,False,False,False,False,False,False,False,False,False,True,False,False,False,2015-09-02,25.3,0.5,10.4,IAH,23.0
881570,2015-03-02,PBI,1743,False,True,False,False,False,False,False,False,False,False,False,False,False,2015-03-02,24.8,0.0,15.1,PBI,1.0
1355755,2015-04-01,DAY,1245,False,False,False,False,False,False,False,False,False,False,False,False,True,2015-04-01,8.2,0.0,12.2,DAY,6.0


In [ ]:
# Fixing departure datetime column
train_data['CRS_DEP_TIME'] = train_data['CRS_DEP_TIME'].apply(lambda x: f"{x:04}").apply(lambda x: f"{x[:2]}:{x[2:]}")
train_data['HOUR'] = pd.to_datetime(train_data['CRS_DEP_TIME'], format = '%H:%M').dt.hour

In [54]:
# Fixing Flight date column and extracting Month and DayofWeek
train_data['FL_DATE'] = pd.to_datetime(train_data['FL_DATE'])
train_data['Month'] = train_data['FL_DATE'].dt.month
train_data['DayOfWeek'] = train_data['FL_DATE'].dt.day_name()

In [56]:
# Performing target encoding
# The order for target encoding is computed from train data to prevent data leakage
# And we will use this same order for test data
# This is useful as we will obtain a linear relationship between the target and predictors

# Aggregating the "DayOfWeek" based on dep_delay to perform target encoding
mean_delay_by_day = train_data.groupby('DayOfWeek')['DEP_DELAY'].mean().sort_values() 
# Using the aggregated data to create a dictionary that contains the rank for the average dep_delay of each dayofweek
encoding_day = {day: rank for rank, day in enumerate(mean_delay_by_day.index, start=1)} 
train_data['DayOfWeek'] = train_data['DayOfWeek'].map(encoding_day)

In [57]:
# Target Encoding for hour(similar to DayOfWeek)
mean_delay_by_hour = train_data.groupby('HOUR')['DEP_DELAY'].mean().sort_values()
encoding_hour = {hour: rank for rank, hour in enumerate(mean_delay_by_hour.index, start=1)}
train_data['HOUR'] = train_data['HOUR'].map(encoding_hour)

In [58]:
# Target Encoding for Month(similar to DayOfWeek)
mean_delay_by_month = train_data.groupby('Month')['DEP_DELAY'].mean().sort_values()
encoding_month = {month: rank for rank, month in enumerate(mean_delay_by_month.index, start=1)}
train_data['Month'] = train_data['Month'].map(encoding_month)

In [59]:
# Removing redundant columns
train_data.drop(['FL_DATE', 'CRS_DEP_TIME', 'ORIGIN', 'Date', 'Airport Code'], axis=1, inplace=True)

In [60]:
train_data.head()

,OP_CARRIER_AS,OP_CARRIER_B6,OP_CARRIER_DL,OP_CARRIER_EV,OP_CARRIER_F9,OP_CARRIER_HA,OP_CARRIER_MQ,OP_CARRIER_NK,OP_CARRIER_OO,OP_CARRIER_UA,OP_CARRIER_US,OP_CARRIER_VX,OP_CARRIER_WN,tavg,prcp,wspd,DEP_DELAY,HOUR,Month,DayOfWeek
1276095,False,True,False,False,False,False,False,False,False,False,False,False,False,26.6,0.0,16.6,16.0,16,5,2
2100329,False,False,False,False,False,False,False,False,False,False,False,False,True,22.7,5.8,9.4,0.0,2,6,7
3830405,False,False,False,False,False,False,False,False,False,True,False,False,False,25.3,0.5,10.4,23.0,5,2,4
881570,False,True,False,False,False,False,False,False,False,False,False,False,False,24.8,0.0,15.1,1.0,19,5,7
1355755,False,False,False,False,False,False,False,False,False,False,False,False,True,8.2,0.0,12.2,6.0,9,4,4


In [62]:
len(train_data[train_data['DEP_DELAY'] > 180])

0

In [63]:
y_train = train_data['DEP_DELAY']
X_train = train_data.drop('DEP_DELAY', axis=1)

### Test

In [64]:
X_test['CRS_DEP_TIME'] = X_test['CRS_DEP_TIME'].apply(lambda x: f"{x:04}").apply(lambda x: f"{x[:2]}:{x[2:]}")
X_test['HOUR'] = pd.to_datetime(X_test['CRS_DEP_TIME'], format = '%H:%M').dt.hour
X_test['FL_DATE'] = pd.to_datetime(X_test['FL_DATE'])
# df_final1['Year'] = df_final1['FL_DATE'].dt.year
X_test['Month'] = X_test['FL_DATE'].dt.month
X_test['DayOfWeek'] = X_test['FL_DATE'].dt.day_name()
X_test['DayOfWeek'] = X_test['DayOfWeek'].map(encoding_day)
X_test['HOUR'] = X_test['HOUR'].map(encoding_hour)
X_test['Month'] = X_test['Month'].map(encoding_month)
X_test.drop(['FL_DATE', 'CRS_DEP_TIME', 'ORIGIN', 'Date', 'Airport Code'], axis=1, inplace=True)

In [65]:
X_test.shape

(477140, 19)

In [66]:
y_test.shape

(477140,)

### Case 2:
### Treating dep_delay to be 0, when the flights take off on time or early.

#### The entire preprocessing is similar as of case 1 with a small difference->
##### df_final1['DEP_DELAY'] = df_final1['DEP_DELAY'].apply(lambda x: x if x > 0 else 0)   ##(Converted dep_delay <= 0 to be 0)

In [7]:
df_final1 = df[['FL_DATE', 'OP_CARRIER', 'ORIGIN', 'CRS_DEP_TIME', 'DEP_DELAY']]
df_final1.dropna(subset='DEP_DELAY', inplace=True)
df_final1['DEP_DELAY'] = df_final1['DEP_DELAY'].apply(lambda x: x if x > 0 else 0)
df_final1 = pd.get_dummies(df_final1, columns=['OP_CARRIER'], drop_first=True)
Airport_Weather['Date'] = pd.to_datetime(Airport_Weather['Date'], format='%Y-%m-%d')
df_final1['FL_DATE'] = pd.to_datetime(df_final1['FL_DATE'], format='%Y-%m-%d')
df_final1 = pd.merge(df_final1, Airport_Weather, left_on=['ORIGIN', 'FL_DATE'], right_on=['Airport Code', 'Date'])

In [10]:
y = df_final1['DEP_DELAY']
X = df_final1.drop('DEP_DELAY', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = X_train.copy()
train_data['DEP_DELAY'] = y_train

In [11]:
train_data['CRS_DEP_TIME'] = train_data['CRS_DEP_TIME'].apply(lambda x: f"{x:04}").apply(lambda x: f"{x[:2]}:{x[2:]}")
train_data['HOUR'] = pd.to_datetime(train_data['CRS_DEP_TIME'], format = '%H:%M').dt.hour
train_data['FL_DATE'] = pd.to_datetime(train_data['FL_DATE'])
# df_final1['Year'] = df_final1['FL_DATE'].dt.year
train_data['Month'] = train_data['FL_DATE'].dt.month
train_data['DayOfWeek'] = train_data['FL_DATE'].dt.day_name()
mean_delay_by_day = train_data.groupby('DayOfWeek')['DEP_DELAY'].mean().sort_values()
encoding_day = {day: rank for rank, day in enumerate(mean_delay_by_day.index, start=1)}
train_data['DayOfWeek'] = train_data['DayOfWeek'].map(encoding_day)
# Encoding for hour
mean_delay_by_hour = train_data.groupby('HOUR')['DEP_DELAY'].mean().sort_values()
encoding_hour = {hour: rank for rank, hour in enumerate(mean_delay_by_hour.index, start=1)}
train_data['HOUR'] = train_data['HOUR'].map(encoding_hour)
mean_delay_by_month = train_data.groupby('Month')['DEP_DELAY'].mean().sort_values()
encoding_month = {month: rank for rank, month in enumerate(mean_delay_by_month.index, start=1)}
train_data['Month'] = train_data['Month'].map(encoding_month)
train_data.drop(['FL_DATE', 'CRS_DEP_TIME', 'ORIGIN', 'Date', 'Airport Code'], axis=1, inplace=True)
train_data.head()

,OP_CARRIER_AS,OP_CARRIER_B6,OP_CARRIER_DL,OP_CARRIER_EV,OP_CARRIER_F9,OP_CARRIER_HA,OP_CARRIER_MQ,OP_CARRIER_NK,OP_CARRIER_OO,OP_CARRIER_UA,OP_CARRIER_US,OP_CARRIER_VX,OP_CARRIER_WN,tavg,prcp,wspd,DEP_DELAY,HOUR,Month,DayOfWeek
1754330,False,False,False,False,False,False,False,False,False,False,False,False,False,28.8,0.0,8.3,0.0,9,4,5
2626127,False,False,False,False,False,False,False,False,True,False,False,False,False,15.6,0.0,14.0,0.0,1,12,1
4157937,False,False,False,False,False,False,False,False,False,False,False,False,False,24.6,0.0,5.8,0.0,16,1,6
1557962,False,False,False,False,False,False,True,False,False,False,False,False,False,12.7,0.0,7.7,0.0,9,4,3
497532,False,False,False,False,False,False,False,False,False,False,True,False,False,17.4,0.0,7.6,0.0,16,11,2


### Regression turned out to be a "Big Failure" as we do not have important features related to air traffic, different types of delays(NAS, Carrier maintenance).
### Better to do classification

# Classification

In [37]:
# Converting the delays into binary:- '<=0' as 0(No delay)
df_final1['DEP_DELAY'] = (df_final1['DEP_DELAY'] > 0).astype(int)

In [39]:
df_final1['DEP_DELAY'].value_counts(normalize=True)*100

DEP_DELAY
0    62.80722
1    37.19278
Name: proportion, dtype: float64

In [41]:
df_final1['CRS_DEP_TIME'] = df_final1['CRS_DEP_TIME'].apply(lambda x: f"{x:04}").apply(lambda x: f"{x[:2]}:{x[2:]}")
df_final1['HOUR'] = pd.to_datetime(df_final1['CRS_DEP_TIME'], format = '%H:%M').dt.hour

In [42]:
df_final1['FL_DATE'] = pd.to_datetime(df_final1['FL_DATE'])
# df_final1['Year'] = df_final1['FL_DATE'].dt.year
df_final1['Month'] = df_final1['FL_DATE'].dt.month
df_final1['DayOfWeek'] = df_final1['FL_DATE'].dt.day_name()

In [44]:
df_final1.head()

,FL_DATE,OP_CARRIER,ORIGIN,CRS_DEP_TIME,DEP_DELAY,Date,tavg,prcp,wspd,Airport Code,HOUR,Month,DayOfWeek
0,2015-01-01,NK,MCO,21:47,0,2015-01-01,18.3,0.0,12.6,MCO,21,1,Thursday
1,2015-01-01,NK,MCO,10:42,0,2015-01-01,18.3,0.0,12.6,MCO,10,1,Thursday
2,2015-01-01,NK,MCO,08:37,1,2015-01-01,18.3,0.0,12.6,MCO,8,1,Thursday
3,2015-01-01,NK,MCO,18:16,0,2015-01-01,18.3,0.0,12.6,MCO,18,1,Thursday
4,2015-01-01,NK,MCO,20:55,0,2015-01-01,18.3,0.0,12.6,MCO,20,1,Thursday


In [48]:
# Performing One Hot Encoding 
df_final1 = pd.get_dummies(df_final1, columns=['OP_CARRIER', 'DayOfWeek', 'HOUR'], drop_first=True)

In [49]:
df_final1.head()

,FL_DATE,ORIGIN,CRS_DEP_TIME,DEP_DELAY,Date,tavg,prcp,wspd,Airport Code,Month,OP_CARRIER_AS,OP_CARRIER_B6,OP_CARRIER_DL,OP_CARRIER_EV,OP_CARRIER_F9,OP_CARRIER_HA,OP_CARRIER_MQ,OP_CARRIER_NK,OP_CARRIER_OO,OP_CARRIER_UA,OP_CARRIER_US,OP_CARRIER_VX,OP_CARRIER_WN,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,HOUR_1,HOUR_2,HOUR_3,HOUR_4,HOUR_5,HOUR_6,HOUR_7,HOUR_8,HOUR_9,HOUR_10,HOUR_11,HOUR_12,HOUR_13,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,2015-01-01,MCO,21:47,0,2015-01-01,18.3,0.0,12.6,MCO,1,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,2015-01-01,MCO,10:42,0,2015-01-01,18.3,0.0,12.6,MCO,1,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2015-01-01,MCO,08:37,1,2015-01-01,18.3,0.0,12.6,MCO,1,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,2015-01-01,MCO,18:16,0,2015-01-01,18.3,0.0,12.6,MCO,1,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,2015-01-01,MCO,20:55,0,2015-01-01,18.3,0.0,12.6,MCO,1,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [50]:
df_final1.drop(['FL_DATE', 'CRS_DEP_TIME', 'ORIGIN', 'Date', 'Airport Code', 'Month'], axis=1, inplace=True)

In [51]:
df_final1.head()

,DEP_DELAY,tavg,prcp,wspd,OP_CARRIER_AS,OP_CARRIER_B6,OP_CARRIER_DL,OP_CARRIER_EV,OP_CARRIER_F9,OP_CARRIER_HA,OP_CARRIER_MQ,OP_CARRIER_NK,OP_CARRIER_OO,OP_CARRIER_UA,OP_CARRIER_US,OP_CARRIER_VX,OP_CARRIER_WN,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,HOUR_1,HOUR_2,HOUR_3,HOUR_4,HOUR_5,HOUR_6,HOUR_7,HOUR_8,HOUR_9,HOUR_10,HOUR_11,HOUR_12,HOUR_13,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,0,18.3,0.0,12.6,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,0,18.3,0.0,12.6,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,1,18.3,0.0,12.6,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0,18.3,0.0,12.6,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,0,18.3,0.0,12.6,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [52]:
df_final1.isnull().sum().sum()

0

In [53]:
df_final1.to_csv('Airline_Final.csv', index=False)

In [54]:
pd.read_csv('Airline_Final.csv')

,DEP_DELAY,tavg,prcp,wspd,OP_CARRIER_AS,OP_CARRIER_B6,OP_CARRIER_DL,OP_CARRIER_EV,OP_CARRIER_F9,OP_CARRIER_HA,OP_CARRIER_MQ,OP_CARRIER_NK,OP_CARRIER_OO,OP_CARRIER_UA,OP_CARRIER_US,OP_CARRIER_VX,OP_CARRIER_WN,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,HOUR_1,HOUR_2,HOUR_3,HOUR_4,HOUR_5,HOUR_6,HOUR_7,HOUR_8,HOUR_9,HOUR_10,HOUR_11,HOUR_12,HOUR_13,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,0,18.3,0.0,12.60000,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,0,18.3,0.0,12.60000,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,1,18.3,0.0,12.60000,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0,18.3,0.0,12.60000,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,0,18.3,0.0,12.60000,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5667509,0,19.5,23.1,9.70000,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5667510,1,19.5,23.1,9.70000,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
5667511,0,12.4,0.0,19.10000,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
5667512,0,16.4,54.4,12.33535,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
